Previous file was getting too chunky. This one has just the finalized complete pipeline for the ml feature engineered thang for 2SLS.

##### Python Libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, StackingRegressor
from sklearn.model_selection import cross_val_predict
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import Ridge
from sklearn.impute import SimpleImputer
import statsmodels.api as sm
from scipy import stats
from xgboost import XGBRegressor
from linearmodels.iv import IV2SLS
import warnings

warnings.filterwarnings('ignore')

Thought vomit
What are my knowns?

I want to show the data initially, and then also show that clicks is endogenous to conversion rate.

Once i'm there I want to show my data isn't that clean and make a cleaned dataset.

So there needs to be a preprocessing step that handles bad data, and logs stuff appropriately.

Now. I want to make the interaction terms and such.

Once that is all done I will create a machine learning instrument from enhanced features.

Then it seems I'm foggy on how the instrument creation and 2sls will work with each other.

Plus the whole point of this is to try to integrate Raj Chetty's 2014 forecast bias with value-added estimates into this somehow.

I want to make sure my entire implementation is proper. Even if the results are null.


##### Preprocessing Start

In [43]:
def clean_data(df):
    """
    Clean and preprocess data before analysis.
    
    Performs:
    1. Handle negative income values
    2. Impute missing income with median
    3. Winsorize income at 1st and 99th percentiles
    4. Filter age to plausible range (10-90 years)
    5. Create logarithmic transformations for skewed variables
    """
    print("\n" + "="*60)
    print("DATA CLEANING AND PREPROCESSING")
    print("="*60)
    
    initial_rows = len(df)
    
    # =====================================================================
    # 1. CLEAN INCOME
    # =====================================================================
    if 'Income' in df.columns:
        # Convert negative income to missing
        neg_income_count = (df['Income'] < 0).sum()
        df.loc[df['Income'] < 0, 'Income'] = np.nan
        
        if neg_income_count > 0:
            print(f"✓ Converted {neg_income_count} negative income values to missing")
        
        # Impute missing income with median
        missing_income = df['Income'].isna().sum()
        if missing_income > 0:
            imputer = SimpleImputer(strategy='median')
            df['Income'] = imputer.fit_transform(df[['Income']])
            print(f"✓ Imputed {missing_income} missing income values with median")
        
        # Winsorize: Cap extremes at 1st and 99th percentile
        lower, upper = df['Income'].quantile([0.01, 0.99])
        income_before = df['Income'].copy()
        df['Income'] = df['Income'].clip(lower, upper)
        winsorized = (income_before != df['Income']).sum()
        print(f"✓ Winsorized {winsorized} income values at 1st/99th percentiles")
        print(f"  Income range: [{lower:,.0f}, {upper:,.0f}]")
    
    # =====================================================================
    # 2. FILTER AGE
    # =====================================================================
    if 'Age' in df.columns:
        age_before = len(df)
        df = df[df['Age'].between(10, 90)]
        age_filtered = age_before - len(df)
        if age_filtered > 0:
            print(f"✓ Filtered {age_filtered} rows with implausible ages (keeping 10-90)")
    
    # =====================================================================
    # 3. CREATE LOGARITHMIC TRANSFORMATIONS
    # =====================================================================
    print(f"\n📊 Creating logarithmic transformations:")
    
    # Log of Income (if positive)
    if 'Income' in df.columns:
        df['Income_log'] = np.log1p(df['Income'])
        print(f"  ✓ Income_log created (log1p transformation)")
    
    # Log of Clicks (if exists and positive)
    if 'Clicks' in df.columns:
        df['Clicks_log'] = np.log1p(df['Clicks'])
        print(f"  ✓ Clicks_log created (log1p transformation)")
    
    # Log of Age (for nonlinear age effects)
    if 'Age' in df.columns:
        df['Age_log'] = np.log1p(df['Age'])
        print(f"  ✓ Age_log created (log1p transformation)")
    
    # Log of CTR (if exists and positive)
    if 'CTR' in df.columns:
        # Ensure CTR is positive before log
        if (df['CTR'] > 0).all():
            df['CTR_log'] = np.log(df['CTR'])
            print(f"  ✓ CTR_log created (log transformation)")
    
    # =====================================================================
    # SUMMARY
    # =====================================================================
    final_rows = len(df)
    rows_removed = initial_rows - final_rows
    
    print(f"\n{'='*60}")
    print(f"CLEANING SUMMARY:")
    print(f"  Initial rows:        {initial_rows:,}")
    print(f"  Final rows:          {final_rows:,}")
    print(f"  Rows removed:        {rows_removed:,} ({rows_removed/initial_rows*100:.1f}%)")
    print(f"  Log variables added: {len([col for col in df.columns if '_log' in col])}")
    print(f"{'='*60}\n")
    
    return df

    
def engineer_time_features(df):
    """Extract day of week and hour from Click_Time"""
    if 'Click_Time' in df.columns:
        df['Click_Time'] = pd.to_datetime(df['Click_Time'])
        df['Day_of_Week'] = df['Click_Time'].dt.dayofweek
        df['Hour'] = df['Click_Time'].dt.hour
    return df
    
def encode_categorical_features(df):
    """Encode categorical variables"""
    categorical_cols = ['Gender', 'Location', 'Ad_Type', 'Ad_Topic', 'Ad_Placement']
    
    for col in categorical_cols:
        if col in df.columns:
            le = LabelEncoder()
            df[f'{col}_encoded'] = le.fit_transform(df[col].astype(str))
            # df[col] = le # This line was wierd, idk why it did this, but i may have to revert things back.
    
    return df

# gonna hold off on this interaction term stuff for now.
def engineer_instrument_features(df):
    """
    ENHANCED: Create rich features that predict clicks but don't directly affect conversions.
    
    This is crucial for instrument strength. We create:
    1. Interaction features between ad characteristics and demographics
    2. Time-based features (weekend, business hours)
    3. Nonlinear transformations
    4. Complex interactions between multiple variables
    
    Key principle: These features should predict CLICKS well, but only affect
    CONVERSIONS through clicks (exclusion restriction).
    """
    print("\n" + "="*60)
    print("FEATURE ENGINEERING FOR INSTRUMENT STRENGTH")
    print("="*60)
    
    # =====================================================================
    # 1. AD CHARACTERISTICS × DEMOGRAPHICS INTERACTIONS
    # =====================================================================
    # Rationale: Different demographics respond differently to ad types
    
    if all(col in df.columns for col in ['Income', 'Ad_Type_encoded']):
        df['Income_x_AdType'] = df['Income'] * df['Ad_Type_encoded']
        print("✓ Created Income × Ad Type interaction")
        
    if all(col in df.columns for col in ['Age', 'Ad_Topic_encoded']):
        df['Age_x_AdTopic'] = df['Age'] * df['Ad_Topic_encoded']
        print("✓ Created Age × Ad Topic interaction")
        
    if all(col in df.columns for col in ['Income', 'Ad_Placement_encoded']):
        df['Income_x_Placement'] = df['Income'] * df['Ad_Placement_encoded']
        print("✓ Created Income × Ad Placement interaction")
        
    if all(col in df.columns for col in ['Age', 'Ad_Placement_encoded']):
        df['Age_x_Placement'] = df['Age'] * df['Ad_Placement_encoded']
        print("✓ Created Age × Ad Placement interaction")
    
    # =====================================================================
    # 2. TIME-BASED FEATURES AND INTERACTIONS
    # =====================================================================
    # Rationale: Click patterns vary by time of day/week
    
    if 'Day_of_Week' in df.columns:
        df['Weekend'] = (df['Day_of_Week'] >= 5).astype(int)
        print("✓ Created Weekend indicator")
        
    if 'Hour' in df.columns:
        df['BusinessHours'] = ((df['Hour'] >= 9) & (df['Hour'] <= 17)).astype(int)
        df['Evening'] = ((df['Hour'] >= 18) & (df['Hour'] <= 23)).astype(int)
        df['Morning'] = ((df['Hour'] >= 6) & (df['Hour'] <= 11)).astype(int)
        print("✓ Created time-of-day indicators")
    
    # Time × Ad interactions
    if all(col in df.columns for col in ['Weekend', 'Ad_Type_encoded']):
        df['Weekend_x_AdType'] = df['Weekend'] * df['Ad_Type_encoded']
        print("✓ Created Weekend × Ad Type interaction")
        
    if all(col in df.columns for col in ['BusinessHours', 'Ad_Placement_encoded']):
        df['BusinessHours_x_Placement'] = df['BusinessHours'] * df['Ad_Placement_encoded']
        print("✓ Created Business Hours × Ad Placement interaction")
        
    if all(col in df.columns for col in ['Evening', 'Ad_Topic_encoded']):
        df['Evening_x_AdTopic'] = df['Evening'] * df['Ad_Topic_encoded']
        print("✓ Created Evening × Ad Topic interaction")
    
    # =====================================================================
    # 3. DEMOGRAPHICS × TIME INTERACTIONS
    # =====================================================================
    # Rationale: Different demographics have different browsing patterns
    
    if all(col in df.columns for col in ['Age', 'Hour']):
        df['Age_x_Hour'] = df['Age'] * df['Hour']
        print("✓ Created Age × Hour interaction")
        
    if all(col in df.columns for col in ['Income', 'Weekend']):
        df['Income_x_Weekend'] = df['Income'] * df['Weekend']
        print("✓ Created Income × Weekend interaction")
        
    if all(col in df.columns for col in ['Age', 'BusinessHours']):
        df['Age_x_BusinessHours'] = df['Age'] * df['BusinessHours']
        print("✓ Created Age × Business Hours interaction")
    
    # =====================================================================
    # 4. NONLINEAR TRANSFORMATIONS
    # =====================================================================
    # Rationale: Relationships may be nonlinear (using log-transformed versions)
    
    if 'Age_log' in df.columns:
        df['Age_squared'] = df['Age'] ** 2
        print("✓ Created Age squared")
        
    if 'Income_log' in df.columns:
        df['Income_squared'] = df['Income'] ** 2
        df['Income_sqrt'] = np.sqrt(df['Income'].clip(lower=0))
        print("✓ Created Income squared and sqrt")
    
    # =====================================================================
    # 5. COMPLEX CATEGORICAL INTERACTIONS
    # =====================================================================
    # Rationale: Certain combinations may be particularly predictive
    
    # Location × Demographics
    if all(col in df.columns for col in ['Location_encoded', 'Age']):
        df['Location_x_Age'] = df['Location_encoded'] * df['Age']
        print("✓ Created Location × Age interaction")
        
    if all(col in df.columns for col in ['Location_encoded', 'Income']):
        df['Location_x_Income'] = df['Location_encoded'] * df['Income']
        print("✓ Created Location × Income interaction")
    
    # Location × Ad characteristics
    if all(col in df.columns for col in ['Location_encoded', 'Ad_Placement_encoded']):
        df['Location_x_Placement'] = df['Location_encoded'] * df['Ad_Placement_encoded']
        print("✓ Created Location × Placement interaction")
    
    # Gender × Ad characteristics
    if all(col in df.columns for col in ['Gender_encoded', 'Ad_Topic_encoded']):
        df['Gender_x_AdTopic'] = df['Gender_encoded'] * df['Ad_Topic_encoded']
        print("✓ Created Gender × Ad Topic interaction")
        
    if all(col in df.columns for col in ['Gender_encoded', 'Ad_Type_encoded']):
        df['Gender_x_AdType'] = df['Gender_encoded'] * df['Ad_Type_encoded']
        print("✓ Created Gender × Ad Type interaction")
    
    # Ad Type × Placement (different placements work for different types)
    if all(col in df.columns for col in ['Ad_Type_encoded', 'Ad_Placement_encoded']):
        df['AdType_x_Placement'] = df['Ad_Type_encoded'] * df['Ad_Placement_encoded']
        print("✓ Created Ad Type × Placement interaction")
    
    # =====================================================================
    # 6. THREE-WAY INTERACTIONS (most powerful)
    # =====================================================================
    # Rationale: Capture complex patterns
    
    if all(col in df.columns for col in ['Age', 'Ad_Type_encoded', 'Weekend']):
        df['Age_x_AdType_x_Weekend'] = df['Age'] * df['Ad_Type_encoded'] * df['Weekend']
        print("✓ Created Age × Ad Type × Weekend interaction")
        
    if all(col in df.columns for col in ['Income', 'Ad_Placement_encoded', 'BusinessHours']):
        df['Income_x_Placement_x_BizHours'] = df['Income'] * df['Ad_Placement_encoded'] * df['BusinessHours']
        print("✓ Created Income × Placement × Business Hours interaction")
    
    print("="*60 + "\n")
    
    return df

##### Preprocessing End

##### Create ML Instrument Start

In [44]:
def create_ml_instrument(df, model_type='stacking', cv_folds=5, use_enhanced_features=False):
    """
    Generate ML-based instrument for Clicks using ensemble methods.
    Returns a new DataFrame with 'Clicks_predicted' column.
    """

    # --- Step 1: Define instrument features (strictly policy-side, not outcomes!) ---
    base_features = [
        'Age', 'Income',
        'Gender_encoded', 'Location_encoded',
        'Ad_Type_encoded', 'Ad_Topic_encoded', 'Ad_Placement_encoded',
        'Day_of_Week', 'Hour'
    ]

    enhanced_features = [
        # Interactions
        'Income_x_AdType', 'Age_x_AdTopic', 'Income_x_Placement', 'Age_x_Placement',
        'Weekend_x_AdType', 'BusinessHours_x_Placement', 'Evening_x_AdTopic',
        'Age_x_Hour', 'Income_x_Weekend', 'Age_x_BusinessHours',
        'Location_x_Age', 'Location_x_Income', 'Location_x_Placement',
        'Gender_x_AdTopic', 'Gender_x_AdType', 'AdType_x_Placement',
        'Age_x_AdType_x_Weekend', 'Income_x_Placement_x_BizHours',
        # Time features
        'Weekend', 'BusinessHours', 'Evening', 'Morning',
        # Nonlinear (now using cleaned log versions) NOTE only created if i log age, and income...
        'Age_squared', 'Age_log', 'Income_log', 'Income_squared', 'Income_sqrt',
        'Clicks_log', 'CTR_log'
    ]

    if use_enhanced_features:
        # Assume you have a separate function to engineer features
        df = engineer_instrument_features(df)
        instrument_features = base_features + enhanced_features
    else:
        instrument_features = base_features

    # Filter available features
    available_features = [f for f in instrument_features if f in df.columns]
    X = df[available_features]
    y = df['Clicks']

    # --- Step 2: Build model ---
    if model_type == 'stacking':
        base_models = [
            ('rf', RandomForestRegressor(n_estimators=200, random_state=42, n_jobs=-1)),
            ('gb', GradientBoostingRegressor(n_estimators=200, random_state=42))
        ]
        try:
            from xgboost import XGBRegressor
            base_models.append(('xgb', XGBRegressor(n_estimators=200, random_state=42, n_jobs=-1)))
        except ImportError:
            pass
        model = StackingRegressor(estimators=base_models, final_estimator=Ridge(alpha=1.0), cv=cv_folds, n_jobs=-1)

    elif model_type == 'rf':
        model = RandomForestRegressor(n_estimators=200, random_state=42, n_jobs=-1)

    elif model_type == 'gb':
        model = GradientBoostingRegressor(n_estimators=200, random_state=42)

    else:
        raise ValueError("Invalid model_type. Choose 'stacking', 'rf', or 'gb'.")

    # --- Step 3: Generate out-of-fold predictions ---
    clicks_pred = cross_val_predict(model, X, y, cv=cv_folds, n_jobs=-1)
    df = df.copy()
    df['Clicks_predicted'] = clicks_pred

    # --- Step 4: Fit final model (optional, for diagnostics) ---
    model.fit(X, y)

    return df, X, model

def enhanced_instrument_diagnostics(df, X, y, model):
    """
    Functional: Comprehensive instrument strength testing with Stock-Yogo critical values.
    
    Parameters
    ----------
    df : pd.DataFrame
        Must contain 'Clicks' and 'Clicks_predicted' columns.
    X : pd.DataFrame
        Feature matrix used in first-stage model.
    y : pd.Series or np.array
        True clicks (endogenous regressor).
    model : fitted sklearn model
        First-stage ML model used to generate instruments.
    """

    # --- Extract instrument (Z) and endogenous regressor (D) ---
    if 'Clicks_predicted' not in df.columns:
        raise ValueError("DataFrame must contain 'Clicks_predicted' column")
    z = df['Clicks_predicted'].values
    d = df['Clicks'].values

    n = len(d)
    k = X.shape[1]

    # --- First-stage R² and F-statistic ---
    d_resid = d - d.mean()
    ss_tot = np.sum(d_resid**2)
    ss_res = np.sum((d - z)**2)
    r_squared = 1 - (ss_res / ss_tot)

    f_stat = (r_squared / 1) / ((1 - r_squared) / (n - k - 1))

    # --- Correlation ---
    corr = np.corrcoef(z, d)[0, 1]

    # --- Cragg-Donald statistic ---
    cragg_donald = n * r_squared

    # --- Display results ---
    print(f"\n{'='*70}")
    print("ENHANCED INSTRUMENT STRENGTH DIAGNOSTICS")
    print(f"{'='*70}")
    print("\nSAMPLE INFORMATION:")
    print(f"  Sample size (n):              {n:,}")
    print(f"  Number of features (k):       {k}")
    print("\nFIRST-STAGE PERFORMANCE:")
    print(f"  R-squared:                    {r_squared:.4f}")
    print(f"  Correlation (Z, D):           {corr:.4f}")
    print(f"  F-statistic:                  {f_stat:.2f}")
    print(f"  Cragg-Donald statistic:       {cragg_donald:.2f}")

    print("\nBENCHMARKS & INTERPRETATION:")
    print(f"  {'Criterion':<35} {'Threshold':<12} {'Status'}")
    print(f"  {'-'*35} {'-'*12} {'-'*20}")
    weak_status = "✓ STRONG" if f_stat > 10 else "✗ WEAK"
    print(f"  {'Weak Instrument (F < 10)':<35} {'10.00':<12} {weak_status}")
    sy_10_status = "✓✓ EXCELLENT" if f_stat > 16.38 else "✗ Below threshold"
    sy_15_status = "✓ GOOD" if f_stat > 8.96 else "✗ Below threshold"
    print(f"  {'Stock-Yogo 10% max bias':<35} {'16.38':<12} {sy_10_status}")
    print(f"  {'Stock-Yogo 15% max bias':<35} {'8.96':<12} {sy_15_status}")

    print("\nOVERALL ASSESSMENT:")
    if f_stat > 16.38:
        print("  ✓✓ VERY STRONG INSTRUMENT")
        print("     Maximum IV bias < 10% of OLS bias")
    elif f_stat > 10:
        print("  ✓ STRONG INSTRUMENT")
        print("     Acceptable for causal inference")
    elif f_stat > 5:
        print("  ⚠ MODERATELY WEAK INSTRUMENT")
        print("     Proceed with caution")
    else:
        print("  ✗ WEAK INSTRUMENT")
        print("     Results may be unreliable")

    # --- Feature importance (if available) ---
    if hasattr(model, 'feature_importances_'):
        print("\nTOP 10 MOST IMPORTANT FEATURES FOR PREDICTING CLICKS:")
        importances = model.feature_importances_
        top_features = sorted(zip(X.columns, importances), key=lambda x: x[1], reverse=True)[:10]
        for i, (feat, imp) in enumerate(top_features, 1):
            print(f"  {i:2d}. {feat:35s} {imp:.4f}")
    elif hasattr(model, 'final_estimator_'):
        print("\nℹ Stacking ensemble used - feature importances not directly available")

    print(f"{'='*70}\n")

##### Create ML Instrument End

##### 2SLS Start

In [49]:
def run_2sls(
    df,
    y_col='Conversion_Rate',
    d_col='Clicks',
    z_col='Clicks_predicted',
    base_controls=None,
    include_interactions=False,
    add_constant=True,
    cluster_col=None,
    cov_type='robust'  # 'robust' for HC, or 'cluster' if cluster_col is set
):
    """
    Functional 2SLS using linearmodels.iv.IV2SLS.

    Model:
        First stage: D = π0 + π1 Z + Γ X + ν
        Second stage: Y = α + β D + Θ X + ε

    Parameters
    ----------
    df : pd.DataFrame
        Must contain y_col, d_col, z_col, and any control columns.
    y_col : str
        Outcome column (e.g., 'Conversion_Rate').
    d_col : str
        Endogenous regressor (e.g., 'Clicks').
    z_col : str
        Instrument column (e.g., 'Clicks_predicted' from ML first stage).
    base_controls : list[str] or None
        Exogenous controls. Do NOT include outcome-adjacent variables like CTR.
        Recommended: ['Age', 'Income', 'Gender_encoded', 'Location_encoded',
                      'Ad_Type_encoded', 'Ad_Topic_encoded', 'Ad_Placement_encoded'].
    include_interactions : bool
        If True, include Ad_Type × Ad_Placement interaction (exogenous).
    add_constant : bool
        If True, add a constant term automatically.
    cluster_col : str or None
        Column name for cluster-robust SEs (e.g., 'UserID', 'CampaignID').
    cov_type : str
        'robust' (HC), 'cluster' (requires cluster_col), or 'unadjusted'.

    Returns
    -------
    results : IV2SLSResults
        Fitted IV results object from linearmodels.
    data_used : pd.DataFrame
        DataFrame with columns actually used in estimation.
    """

    # --- Validate required columns ---
    required = [y_col, d_col, z_col]
    missing = [c for c in required if c not in df.columns]
    if missing:
        raise ValueError(f"Missing required columns: {missing}")

    # --- Controls: enforce exogeneity discipline ---
    if base_controls is None:
        base_controls = [
            'Age', 'Income',
            'Gender_encoded', 'Location_encoded',
            'Ad_Type_encoded', 'Ad_Topic_encoded', 'Ad_Placement_encoded'
        ]

    # Strict: exclude CTR or any post-click/outcome-adjacent metrics from controls
    invalid_controls = [c for c in base_controls if c not in df.columns]
    if invalid_controls:
        # Warn strictly but proceed with available subset
        print(f"⚠ Skipping missing controls: {invalid_controls}")
    controls = [c for c in base_controls if c in df.columns]

    # --- Optional exogenous interaction ---
    interaction_col = None
    if include_interactions:
        if ('Ad_Type_encoded' in df.columns) and ('Ad_Placement_encoded' in df.columns):
            interaction_col = 'Ad_Type_x_Placement'
            if interaction_col not in df.columns:
                df = df.copy()
                df[interaction_col] = df['Ad_Type_encoded'] * df['Ad_Placement_encoded']
                print("✓ Added exogenous interaction: Ad_Type_x_Placement")
            controls.append(interaction_col)
        else:
            print("⚠ Interaction requested but required columns not present; skipping.")

    # --- Build data frame used in estimation ---
    cols_needed = [y_col, d_col, z_col] + controls
    data = df[cols_needed].dropna().copy()
    if data.empty:
        raise ValueError("After dropping NA, no rows remain for estimation.")

    # --- Build formula for IV2SLS ---
    # dependent ~ exog + [endog ~ instruments]
    exog_formula = ' + '.join(controls) if controls else '1'
    if add_constant and exog_formula != '1':
        exog_formula = '1 + ' + exog_formula  # linearmodels adds constant via '1 +'
    elif add_constant and exog_formula == '1':
        # '1' already denotes constant in linearmodels formula
        pass
    else:
        # No constant: use '-1' to suppress intercept if you have exog terms
        if controls:
            exog_formula = '-1 + ' + ' + '.join(controls)

    formula = f"{y_col} ~ {exog_formula} + [{d_col} ~ {z_col}]"

    # --- Fit IV2SLS ---
    if cov_type == 'cluster' and (cluster_col is not None) and (cluster_col in df.columns):
        clusters = df.loc[data.index, cluster_col]
        results = IV2SLS.from_formula(formula, data=data).fit(cov_type='clustered', clusters=clusters)
    else:
        if cov_type == 'cluster' and cluster_col is None:
            print("⚠ cov_type='cluster' requested but no cluster_col provided; defaulting to robust.")
        results = IV2SLS.from_formula(formula, data=data).fit(cov_type='robust' if cov_type != 'unadjusted' else 'unadjusted')

    # --- Strict reporting ---
    print("\n" + "="*70)
    print("2SLS ESTIMATION SUMMARY (linearmodels.iv.IV2SLS)")
    print("="*70)
    print(results.summary)

    # Optional first-stage diagnostics available via .first_stage (dict of RegressionResults)
    # Example:
    try:
        fs = results.first_stage[d_col]
        print("\nFirst-stage summary (endogenous regressor: {}):".format(d_col))
        print(f"  R-squared: {fs.rsquared:.4f}")
        print(f"  F-statistic (excluded instrument): {getattr(fs, 'f_statistic', None)}")
    except Exception:
        pass

    return results, data

##### 2SLS End

In [ ]:
def generate_example_data(n=2000):
    """Generate synthetic data for demonstration"""
    np.random.seed(42)
    
    data = pd.DataFrame({
        'Age': np.random.randint(18, 65, n),
        'Gender': np.random.choice(['M', 'F'], n),
        'Income': np.random.randint(30000, 150000, n),
        'Location': np.random.choice(['Urban', 'Suburban', 'Rural'], n),
        'Ad_Type': np.random.choice(['Video', 'Banner', 'Native'], n),
        'Ad_Topic': np.random.choice(['Tech', 'Fashion', 'Food', 'Travel'], n),
        'Ad_Placement': np.random.choice(['Social_Media', 'Search', 'Display'], n),
        'Click_Time': pd.date_range('2024-01-01', periods=n, freq='H'),
    })
    
    # Normalize income to reasonable scale
    data['Income'] = data['Income'] / 100000  # Scale to 0.3-1.5 range
    
    # Generate clicks with realistic structure
    clicks_base = (
        0.5 +  # baseline
        0.3 * (data['Ad_Type'] == 'Video').astype(float) +
        0.2 * (data['Ad_Placement'] == 'Social_Media').astype(float) +
        0.01 * data['Age'] +
        0.2 * data['Income'] +
        np.random.randn(n) * 0.5
    )
    data['Clicks'] = np.clip(clicks_base, 0.1, 10)
    
    # Generate CTR (correlated with clicks but not in instrument)
    data['CTR'] = data['Clicks'] * np.random.uniform(0.05, 0.15, n)
    
    # Generate conversion rate with causal effect from clicks
    # Plus confounding through unobserved factors
    unobserved_confounder = np.random.randn(n) * 0.05
    
    conversion_base = (
        0.05 +  # baseline
        0.08 * data['Clicks'] +  # TRUE CAUSAL EFFECT
        0.02 * data['Income'] +
        0.005 * data['Age'] +
        0.3 * data['CTR'] +
        unobserved_confounder +
        np.random.randn(n) * 0.03
    )
    data['Conversion_Rate'] = np.clip(conversion_base, 0.01, 0.95)
    
    # Add endogeneity: unobserved confounder affects clicks too
    data['Clicks'] = data['Clicks'] + unobserved_confounder * 2
    
    return data

##### Start of Implementation

In [55]:
# Cleaning and preprocessing
# df = pd.read_csv('../datasets/project/Dataset_Ads.csv')
df = generate_example_data(n=5000)
print("\n" + "="*60)
print('ORIGINAL DATASET')
print("="*60)
print(df.head())

df = clean_data(df)
print("\n" + "="*60)
print('CLEANED AND LOGGED DATASET')
print("="*60)
print(df.head())

df = engineer_time_features(df)
print("\n" + "="*60)
print('TIME ENGINEERED COLUMN')
print("="*60)
print(df.head())

df = encode_categorical_features(df)
print("\n" + "="*60)
print('ENCODED CATEGORICAL VARIABLES')
print("="*60)
print(df.head())

print("\n" + "="*60)
print('DESCRIPTION OF DF AFTER PREPROCESSING')
print("="*60)
print(df.describe(include='all'))


ORIGINAL DATASET
   Age Gender   Income  Location Ad_Type Ad_Topic  Ad_Placement  \
0   56      M  0.87676  Suburban  Native   Travel        Search   
1   46      F  0.41560     Rural  Native   Travel       Display   
2   32      M  0.84401     Rural  Banner     Tech       Display   
3   60      M  0.80468  Suburban  Native     Food  Social_Media   
4   25      M  1.45884     Rural  Native  Fashion       Display   

           Click_Time    Clicks       CTR  Conversion_Rate  
0 2024-01-01 00:00:00  0.607039  0.072258         0.390314  
1 2024-01-01 01:00:00  0.501653  0.063577         0.271384  
2 2024-01-01 02:00:00  1.000099  0.073307         0.322355  
3 2024-01-01 03:00:00  1.599193  0.163070         0.535647  
4 2024-01-01 04:00:00  2.066244  0.179659         0.373919  

DATA CLEANING AND PREPROCESSING
✓ Winsorized 100 income values at 1st/99th percentiles
  Income range: [0, 1]

📊 Creating logarithmic transformations:
  ✓ Income_log created (log1p transformation)
  ✓ Clicks_log 

In [ ]:
# ML instrument creation took about 1m40secs with real data
# ML instrument creation took about 3m8secs with synthetic data
print("\n" + "="*60)
print('CREATING ML INSTRUMENT')
print("="*60)
df, X, model = create_ml_instrument(df, use_enhanced_features=True)
# NOTE Remember that the instrument is really weak when created not using the interaction terms.


CREATING ML INSTRUMENT

FEATURE ENGINEERING FOR INSTRUMENT STRENGTH
✓ Created Income × Ad Type interaction
✓ Created Age × Ad Topic interaction
✓ Created Income × Ad Placement interaction
✓ Created Age × Ad Placement interaction
✓ Created Weekend indicator
✓ Created time-of-day indicators
✓ Created Weekend × Ad Type interaction
✓ Created Business Hours × Ad Placement interaction
✓ Created Evening × Ad Topic interaction
✓ Created Age × Hour interaction
✓ Created Income × Weekend interaction
✓ Created Age × Business Hours interaction
✓ Created Age squared
✓ Created Income squared and sqrt
✓ Created Location × Age interaction
✓ Created Location × Income interaction
✓ Created Location × Placement interaction
✓ Created Gender × Ad Topic interaction
✓ Created Gender × Ad Type interaction
✓ Created Ad Type × Placement interaction
✓ Created Age × Ad Type × Weekend interaction
✓ Created Income × Placement × Business Hours interaction



In [61]:
# diagnostics for ml instrument strength
print("\n" + "="*60)
print('ML INSTRUMENT DIAGNOSTICS')
print("="*60)
enhanced_instrument_diagnostics(df, X, df['Clicks'], model)


ML INSTRUMENT DIAGNOSTICS

ENHANCED INSTRUMENT STRENGTH DIAGNOSTICS

SAMPLE INFORMATION:
  Sample size (n):              5,000
  Number of features (k):       38

FIRST-STAGE PERFORMANCE:
  R-squared:                    0.9998
  Correlation (Z, D):           0.9999
  F-statistic:                  30484680.67
  Cragg-Donald statistic:       4999.19

BENCHMARKS & INTERPRETATION:
  Criterion                           Threshold    Status
  ----------------------------------- ------------ --------------------
  Weak Instrument (F < 10)            10.00        ✓ STRONG
  Stock-Yogo 10% max bias             16.38        ✓✓ EXCELLENT
  Stock-Yogo 15% max bias             8.96         ✓ GOOD

OVERALL ASSESSMENT:
  ✓✓ VERY STRONG INSTRUMENT
     Maximum IV bias < 10% of OLS bias

ℹ Stacking ensemble used - feature importances not directly available



In [ ]:
# 2SLS with ml featured instrument
# The r-squared value is super tiny with real data for whatever reason.
# However, with my synthetic data I actually get a good r-squared score and a small p-value.
# Perhpas the move is to end the real data stuff there, but continue with Raj Chetty methodlogies with the
# synthetic data for reaserch sake.

controls = [
    'Age', 'Income',
    'Gender_encoded', 'Location_encoded',
    'Ad_Type_encoded', 'Ad_Topic_encoded', 'Ad_Placement_encoded'
]

results, data_used = run_2sls(
    df,
    y_col='Conversion_Rate',
    d_col='Clicks',
    z_col='Clicks_predicted',
    base_controls=controls,
    include_interactions=True,   # optional
    add_constant=True,
    cov_type='robust'            # or 'cluster' with cluster_col='CampaignID'
)

✓ Added exogenous interaction: Ad_Type_x_Placement

2SLS ESTIMATION SUMMARY (linearmodels.iv.IV2SLS)
                          IV-2SLS Estimation Summary                          
Dep. Variable:        Conversion_Rate   R-squared:                      0.8133
Estimator:                    IV-2SLS   Adj. R-squared:                 0.8130
No. Observations:                5000   F-statistic:                 2.179e+04
Date:                Wed, Nov 12 2025   P-value (F-stat)                0.0000
Time:                        11:47:13   Distribution:                  chi2(9)
Cov. Estimator:                robust                                         
                                                                              
                                  Parameter Estimates                                   
                      Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
----------------------------------------------------------------------------------------


##### End of Implementation